# AI Pair Programming

Dans ce notebook nous allons demander à une intelligence artificielle de générer du code Python pour réaliser des tâches de traitement automatique de corpus.

Avant de commencer, choisissez un outil comme [Bard](https://bard.google.com/u/2/chat) ou [ChatGPT](https://chat.openai.com/) et créez un compte.

Vous pouvez ensuite demander à l'outil de créer du code. Avant de commencer, n'hésitez pas à lire [cet article](https://exocoding.com/ai-code-generation/) qui détaille les **bonnes pratiques** pour créer des _prompts_ efficaces dans le cadre de la génération de code par l'intelligence artificielle.


### 1. Algorithme simple en Python

Demandez à l'IA de générer un code python qui lance un décompte du réveillon du Nouvel An. Le code doit imprimer les nombres de 10 à 0 avec un intervalle d'une seconde, puis imprimer "Bonne année" à la fin.

In [ ]:
import time

print("Décompte du Nouvel An !")
for i in range(10, 0, -1):
    print(i)
    time.sleep(1)

print("Bonne année !")

Décompte du Nouvel An !
10
9
8
7
6
5
4
3
2
1
Bonne année !


: 

### 2. Détection du sujet de la phrase

Demandez à l'IA d'extraire le sujet dans une phrase.
Demandez ensuite de générer le code Python qui réalise cette tâche et testez le ci-dessous.

In [4]:
import spacy

# Chargement du modèle français (small/sm est suffisant pour l'analyse de dépendance)
try:
    nlp = spacy.load("fr_core_news_sm")
except OSError:
    print("Erreur : Le modèle 'fr_core_news_sm' n'a pas pu être chargé.")
    print("Assurez-vous d'avoir exécuté : 'python -m spacy download fr_core_news_sm'")
    exit()

def extraire_sujet(phrase: str) -> str:
    """
    Analyse la phrase en utilisant SpaCy et extrait le sujet grammatical.
    
    Args:
        phrase (str): La phrase à analyser.
        
    Returns:
        str: Le sujet trouvé ou un message si aucun n'est identifié.
    """
    
    # 1. Traitement de la phrase par le pipeline SpaCy
    doc = nlp(phrase)
    
    sujets_trouves = []
    
    # 2. Parcourir chaque mot (jeton) dans le document
    for token in doc:
        # 3. Vérifier si la relation de dépendance (dep_) est 'nsubj' (sujet nominal)
        # Et s'assurer que le mot est un nom (NOUN), un pronom (PRON) ou un nom propre (PROPN)
        # Le 'sujet' est identifié par le tag 'nsubj'
        if token.dep_ == "nsubj":
            # Le sujet peut être un groupe de mots (e.g., "Le petit chat")
            # Pour capturer le groupe nominal complet, on récupère son "subtree" (sous-arbre)
            sujet_complet = " ".join([descendant.text for descendant in token.subtree])
            sujets_trouves.append(sujet_complet)
            
            # Dans la plupart des cas, on cherche un seul sujet principal. On s'arrête là.
            break 
            
    if sujets_trouves:
        # Retourne le premier sujet trouvé (le plus probable sujet principal)
        return sujets_trouves[0]
    else:
        return "Sujet non trouvé (ou la phrase est impérative/passive complexe)."

# ==============================================================================
#                                   TESTS
# ==============================================================================

phrases_test = [
    "Le chat noir dort sur le canapé rouge.",
    "Qui est cette personne ici ?",
    "La directrice des ventes a pris une décision importante mais quand même...",
    "Il pleut dehors.", # Sujet impersonnel, souvent marqué 'nsubj'
    "Mange ta soupe !" # Phrase impérative sans sujet explicite
]

print("--- Extraction du Sujet Grammatical ---")

for phrase in phrases_test:
    sujet = extraire_sujet(phrase)
    print(f"\nPhrase : **{phrase}**")
    print(f"Sujet extrait : `{sujet}`")


--- Extraction du Sujet Grammatical ---

Phrase : **Le chat noir dort sur le canapé rouge.**
Sujet extrait : `Sujet non trouvé (ou la phrase est impérative/passive complexe).`

Phrase : **Qui est cette personne ici ?**
Sujet extrait : `Qui`

Phrase : **La directrice des ventes a pris une décision importante mais quand même...**
Sujet extrait : `La directrice des ventes`

Phrase : **Il pleut dehors.**
Sujet extrait : `Sujet non trouvé (ou la phrase est impérative/passive complexe).`

Phrase : **Mange ta soupe !**
Sujet extrait : `Mange`


### 3. Entités nommmées liées à Wikidata

Demandez à l'IA d'extraire les entités nommées d'un texte en français, et de les lier à un identifiant wikidata.
Demandez ensuite à l'IA de générer le code Python pour réaliser cette tâche et testez le ci-dessous.

#Étapes de préparation, en utilisant le préfixe ! si vous êtes dans un Notebook :

In [1]:
pip install spacy pylinks

Note: you may need to restart the kernel to use updated packages.


In [2]:
!python -m spacy download fr_core_news_lg

     ---------------------------------------- 0.0/571.8 MB ? eta -:--:--
     ---------------------------------------- 0.3/571.8 MB ? eta -:--:--
     ---------------------------------------- 0.8/571.8 MB 3.3 MB/s eta 0:02:52
     ---------------------------------------- 1.3/571.8 MB 2.7 MB/s eta 0:03:33
     ---------------------------------------- 1.6/571.8 MB 2.1 MB/s eta 0:04:26
     ---------------------------------------- 1.8/571.8 MB 2.1 MB/s eta 0:04:38
     ---------------------------------------- 2.1/571.8 MB 1.9 MB/s eta 0:05:01
     ---------------------------------------- 2.6/571.8 MB 1.8 MB/s eta 0:05:10
     ---------------------------------------- 2.9/571.8 MB 1.9 MB/s eta 0:05:02
     ---------------------------------------- 3.4/571.8 MB 1.9 MB/s eta 0:04:57
     ---------------------------------------- 3.9/571.8 MB 2.0 MB/s eta 0:04:46
     ---------------------------------------- 4.7/571.8 MB 2.1 MB/s eta 0:04:31
     ---------------------------------------- 5.2/571.

In [3]:
!pip install spacy pylinks spacy-entity-linker

In [ ]:
from spacy.kb import InMemoryLookupKB 

# Remplace KnowledgeBase par InMemoryLookupKB en Windows
kb = InMemoryLookupKB(vocab=nlp.vocab, entity_vector_length=64) 


In [13]:
import spacy
import pandas as pd
import requests
from typing import Optional

Première tentative de éxecuter le code généré par l'IA :


In [14]:
def rechercher_wikidata_id(nom_entite: str, langue: str = "fr") -> Optional[str]:
    """
    Recherche l'ID Wikidata correspondant à une entité via l'API Wikidata.
    """
    url = "https://www.wikidata.org/w/api.php"
    params = {
        'action': 'wbsearchentities',
        'search': nom_entite,
        'language': langue,
        'format': 'json'
    }
    
    try:
        response = requests.get(url, params=params, timeout=10)
        data = response.json()
        
        if data.get('search'):
            # Retourner l'ID du premier résultat
            return data['search'][0]['id']
    except Exception as e:
        print(f"Erreur lors de la recherche Wikidata pour '{nom_entite}': {e}")
    
    return None

def extraire_et_lier_entites_wikidata(texte):
    """
    Extrait les entités et les lie à Wikidata via l'API.
    """
    # Charger le modèle spaCy
    nlp = spacy.load("fr_core_news_sm")
    
    # Traitement du texte
    doc = nlp(texte)
    
    resultats = []
    
    for ent in doc.ents:
        wikidata_id = rechercher_wikidata_id(ent.text)
        
        resultats.append({
            'entite': ent.text,
            'label': ent.label_,
            'description': spacy.explain(ent.label_),
            'wikidata_id': wikidata_id,
            'debut': ent.start_char,
            'fin': ent.end_char
        })
    
    return pd.DataFrame(resultats)

# Test final avec la solution recommandée
texte_test = "Emmanuel Macron est le président de la France. Paris est la capitale."
df_resultats = extraire_et_lier_entites_wikidata(texte_test)

if not df_resultats.empty:
    print("--- Résultats de l'extraction et du linking ---")
    print(df_resultats)
else:
    print("Aucune entité trouvée.")

Erreur lors de la recherche Wikidata pour 'Emmanuel Macron': Expecting value: line 1 column 1 (char 0)
Erreur lors de la recherche Wikidata pour 'la France': Expecting value: line 1 column 1 (char 0)
Erreur lors de la recherche Wikidata pour 'Paris': Expecting value: line 1 column 1 (char 0)
--- Résultats de l'extraction et du linking ---
            entite label                                        description  \
0  Emmanuel Macron   PER                            Named person or family.   
1        la France   LOC  Non-GPE locations, mountain ranges, bodies of ...   
2            Paris   LOC  Non-GPE locations, mountain ranges, bodies of ...   

  wikidata_id  debut  fin  
0        None      0   15  
1        None     36   45  
2        None     47   52  


Deuxième tentative de éxecuter le code généré par l'IA :

In [16]:
def rechercher_wikidata_id(nom_entite: str, langue: str = "fr") -> Optional[str]:
    """
    Recherche l'ID Wikidata correspondant à une entité via l'API Wikidata.
    """
    url = "https://www.wikidata.org/w/api.php"
    params = {
        'action': 'wbsearchentities',
        'search': nom_entite,
        'language': langue,
        'format': 'json'
    }
    
    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()  # Lève une exception pour les codes d'erreur HTTP
        data = response.json()
        
        if data.get('search'):
            # Retourner l'ID du premier résultat
            return data['search'][0]['id']
        else:
            print(f"Aucun résultat Wikidata pour '{nom_entite}'")
    except requests.exceptions.RequestException as e:
        print(f"Erreur de requête pour '{nom_entite}': {e}")
    except ValueError as e:
        print(f"Erreur de décodage JSON pour '{nom_entite}': {e}")
        print(f"Contenu de la réponse: {response.text}")
    except Exception as e:
        print(f"Erreur inattendue pour '{nom_entite}': {e}")
    
    return None

In [17]:
def nettoyer_entite(entite: str) -> str:
    articles = ['le ', 'la ', 'les ', 'l\'', 'un ', 'une ', 'des ']
    entite_nettoyee = entite
    for article in articles:
        if entite_nettoyee.lower().startswith(article):
            entite_nettoyee = entite_nettoyee[len(article):]
            break
    return entite_nettoyee

In [18]:
import spacy
import pandas as pd
import requests
from typing import Optional

def rechercher_wikidata_id(nom_entite: str, langue: str = "fr") -> Optional[str]:
    """
    Recherche l'ID Wikidata correspondant à une entité via l'API Wikidata.
    """
    url = "https://www.wikidata.org/w/api.php"
    params = {
        'action': 'wbsearchentities',
        'search': nom_entite,
        'language': langue,
        'format': 'json'
    }
    
    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()  # Lève une exception pour les codes d'erreur HTTP
        data = response.json()
        
        if data.get('search'):
            # Retourner l'ID du premier résultat
            return data['search'][0]['id']
        else:
            print(f"Aucun résultat Wikidata pour '{nom_entite}'")
    except requests.exceptions.RequestException as e:
        print(f"Erreur de requête pour '{nom_entite}': {e}")
    except ValueError as e:
        print(f"Erreur de décodage JSON pour '{nom_entite}': {e}")
        print(f"Contenu de la réponse: {response.text}")
    except Exception as e:
        print(f"Erreur inattendue pour '{nom_entite}': {e}")
    
    return None

def extraire_et_lier_entites_wikidata(texte):
    """
    Extrait les entités et les lie à Wikidata via l'API.
    """
    # Charger le modèle spaCy
    nlp = spacy.load("fr_core_news_sm")
    
    # Traitement du texte
    doc = nlp(texte)
    
    resultats = []
    
    for ent in doc.ents:
        wikidata_id = rechercher_wikidata_id(ent.text)
        
        resultats.append({
            'entite': ent.text,
            'label': ent.label_,
            'description': spacy.explain(ent.label_),
            'wikidata_id': wikidata_id,
            'debut': ent.start_char,
            'fin': ent.end_char
        })
    
    return pd.DataFrame(resultats)

# Test final avec la solution recommandée
texte_test = "Emmanuel Macron est le président de la France. Paris est la capitale."
df_resultats = extraire_et_lier_entites_wikidata(texte_test)

if not df_resultats.empty:
    print("--- Résultats de l'extraction et du linking ---")
    print(df_resultats)
else:
    print("Aucune entité trouvée.")

Erreur de requête pour 'Emmanuel Macron': 403 Client Error: Forbidden for url: https://www.wikidata.org/w/api.php?action=wbsearchentities&search=Emmanuel+Macron&language=fr&format=json
Erreur de requête pour 'la France': 403 Client Error: Forbidden for url: https://www.wikidata.org/w/api.php?action=wbsearchentities&search=la+France&language=fr&format=json
Erreur de requête pour 'Paris': 403 Client Error: Forbidden for url: https://www.wikidata.org/w/api.php?action=wbsearchentities&search=Paris&language=fr&format=json
--- Résultats de l'extraction et du linking ---
            entite label                                        description  \
0  Emmanuel Macron   PER                            Named person or family.   
1        la France   LOC  Non-GPE locations, mountain ranges, bodies of ...   
2            Paris   LOC  Non-GPE locations, mountain ranges, bodies of ...   

  wikidata_id  debut  fin  
0        None      0   15  
1        None     36   45  
2        None     47   52  


### 4. A vous de jouer

Pensez à une analyse que vous voudriez faire sur un texte. Demandez à l'IA de générer un code python qui réalise cette analyse et testez le ci-dessous

#Gemini, genere un script pour analyser les fichiers PDF et extraire le mot amour de chaque page, puis sauvegarder le texte dans un fichier .txt

In [1]:
pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [3]:
import re
from pathlib import Path
from pypdf import PdfReader
import os

# ==============================================================================
#                 CONFIGURATION À MODIFIER
# ==============================================================================

# 1. Définissez le mot à rechercher (la recherche est non sensible à la casse)
MOT_RECHERCHE = "Bruxelles"

# 2. Définissez le CHEMIN D'ACCÈS VERS VOTRE DOSSIER.
# Exemple : si vos PDF sont sur votre bureau, utilisez Path.home() / "Desktop" / "Mon_Dossier_PDF"
# REMPLACEZ 'chemin/vers/votre/dossier' par le chemin réel.
DOSSIER_PDF = "C:/Users/aspng/ULB_TAC/tac/data/txt" 

# ==============================================================================

def analyser_dossier_pdf(dossier_path, mot):
    """
    Parcourt un dossier, analyse chaque fichier PDF et compte
    les occurrences du mot spécifié.
    """
    total_occurrences = 0
    mot_lower = mot.lower()
    dossier = Path(dossier_path)

    if not dossier.is_dir():
        print(f"❌ Erreur : Le dossier spécifié n'existe pas ou n'est pas un répertoire : {dossier_path}")
        return

    print(f"🔎 Analyse des fichiers PDF dans le répertoire : {dossier_path}")
    print(f"Mot recherché : '{mot}' (recherche non sensible à la casse)\n")
    
    # Chercher tous les fichiers .pdf dans le dossier
    for fichier_pdf in dossier.glob('*.pdf'):
        try:
            print(f"-> Analyse du fichier : {fichier_pdf.name}")
            
            reader = PdfReader(fichier_pdf)
            compteur_fichier = 0
            
            # Parcourir chaque page du document
            for page in reader.pages:
                text = page.extract_text()
                
                if text:
                    text_lower = text.lower()
                    
                    # Utilisation d'une expression régulière (\b) pour trouver
                    # le mot exact (exclut "amoureux", "amourette", etc.)
                    # re.escape est utilisé pour gérer les caractères spéciaux
                    pattern = r'\b' + re.escape(mot_lower) + r'\b'
                    occurrences_page = len(re.findall(pattern, text_lower))
                    
                    compteur_fichier += occurrences_page
            
            total_occurrences += compteur_fichier
            print(f"   ✅ Nombre d'{mot}' trouvés dans ce fichier : {compteur_fichier}")

        except Exception as e:
            print(f"   ❌ ERREUR lors de l'analyse du fichier {fichier_pdf.name} : {e}")

    # Afficher le résultat final
    print("\n" + "="*70)
    print(f"🌟 RÉSULTAT FINAL :")
    print(f"Le mot '{mot}' apparaît un total de {total_occurrences} fois dans tous les documents PDF.")
    print("="*70)

# Exécution du script
if __name__ == "__main__":
    analyser_dossier_pdf(DOSSIER_PDF, MOT_RECHERCHE)

🔎 Analyse des fichiers PDF dans le répertoire : C:/Users/aspng/ULB_TAC/tac/data/txt
Mot recherché : 'Bruxelles' (recherche non sensible à la casse)


🌟 RÉSULTAT FINAL :
Le mot 'Bruxelles' apparaît un total de 0 fois dans tous les documents PDF.



### Pour aller plus loin...

En tant qu'étudiant ULB vous avez accès gratuitement au [Github student pack](https://education.github.com/pack). Ce pack vous donne permet d'utiliser [Github copilot](https://github.com/features/copilot), un outil d'auto-complétion de code grâce à l'intelligence artificielle. Ceci peut être très utile si vous voulez réaliser des tâches complexes sans être un expert en python.